# Анализ базы данных сервиса для чтения книг по подписке

1. [Введение](#introduction)
2. [Загрузка данных](#getting)
3. [Анализ данных](#analys)
* [Посчитаем, сколько книг вышло после 1 января 2000 года](#task_one)
* [Для каждой книги посчитаем количество обзоров и среднюю оценку](#task_two)
* [Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры
](#task_three)
* [Определим  автора с самой высокой средней оценкой книг , будем  учитывать только книги с 50 и более оценками](#task_four)
* [Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок](#task_five)
4. [Вывод](#conclusion)

<a id="introduction"></a> 
### Введение

__Актуальность темы исследования.__ Коронавирус застал мир врасплох, изменив привычный порядок вещей. На какое-то время жители городов перестали выходить на улицу, посещать кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Наша первая задача как аналитика — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

__Цель проекта:__ провести анализ базы данных сервиса для чтения книг по подписке.

__Задачи__ проекта:
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

__Объект__ исследования: сервис для чтения книг по подписке.

__Предмет__ исследования: базы данных сервиса для чтения книг по подписке.


__Структура БД__   :

1. Таблица books содержит данные о книгах:

* book_id — идентификатор книги;
* author_id — идентификатор автора;
* title — название книги;
* num_pages — количество страниц;
* publication_date — дата публикации книги;
* publisher_id — идентификатор издателя.

2. Таблица authors содержит данные об авторах:

* author_id — идентификатор автора;
* author — имя автора.

3. Таблица publishers содержит данные об издательствах:

* publisher_id — идентификатор издательства;
* publisher — название издательства;

4. Таблица ratings содержит данные о пользовательских оценках книг:

* rating_id — идентификатор оценки;
* book_id — идентификатор книги;
* username — имя пользователя, оставившего оценку;
* rating — оценка книги.

5. Таблица reviews содержит данные о пользовательских обзорах:

* review_id — идентификатор обзора;
* book_id — идентификатор книги;
* username — имя автора обзора;
* text — текст обзора.

    
В дорожную карту проекта включили следующие этапы:
1. Загрузка, первичный осмотр данных
2. Проведение анализа данных
3. Выводы

<a id="getting"></a> 
### Загрузим данные и исследуем таблицы 

In [15]:
# импортируем библиотеки 
import pandas as pd 
from sqlalchemy import text, create_engine 

In [16]:
pd.set_option('display.max_columns', None)

In [17]:
# устанавливаем параметры 
db_config = {'user': 'praktikum_student',                     # имя пользователя 
             'pwd':  'Sdf4$2;d-d30pp',                        # пароль 
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', 
             'port':  6432,                                   # порт подключения 
             'db':   'data-analyst-final-project-db'}         # название базы данных 
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config) 
 
# сохраняем коннектор 
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [18]:
table_name = ['books', 'authors', 'ratings', 'reviews', 'publishers']
for name in table_name:
    query_one = f'''SELECT * FROM {name} LIMIT 5''' 
    con=engine.connect() 
    df = pd.io.sql.read_sql(sql=text(query_one), con = con)
    print(f'Таблица {name}')
    print()
    display(df)
    print('*'*50)

Таблица books



,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**************************************************
Таблица authors



,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**************************************************
Таблица ratings



...

**************************************************
Таблица reviews



,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**************************************************
Таблица publishers



,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**************************************************


<a id="analys"></a> 
### Анализ данных

<a id="task_one"></a> 
#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [10]:
query_one = '''SELECT COUNT(book_id) as book_count 
               FROM   books 
               WHERE  (cast(publication_date as date) > '2000-01-01') 
            ''' 

con=engine.connect() 
pd.io.sql.read_sql(sql=text(query_one), con = con)


,book_count
0,819


После 1 января 2000 года вышло 819 книг

<a id="task_two"></a> 
#### Для каждой книги посчитаем количество обзоров и среднюю оценку

In [11]:
query_one = '''with 
               one as (SELECT    book_id,
                                 COUNT(review_id) as review_count
                       FROM      reviews
                       GROUP BY  book_id
                       ORDER BY  review_count DESC),
             
               two as (SELECT    book_id,
                                 avg(rating) as rating_avg
                       FROM      ratings
                       GROUP BY  book_id
                       ORDER BY  rating_avg DESC)
               SELECT     b.title as title,
                          one.review_count as review,
                          ROUND(two.rating_avg,2) as rating
               FROM       books as b
               LEFT JOIN one on one.book_id = b.book_id           
               LEFT JOIN  two on two.book_id = b.book_id           
               ORDER BY   rating desc,
                          review desc,
                          title
               LIMIT      10
             ''' 
con=engine.connect() 
pd.io.sql.read_sql(sql=text(query_one), con = con)

,title,review,rating
0,A Dirty Job (Grim Reaper #1),4,5.0
1,Moneyball: The Art of Winning an Unfair Game,3,5.0
2,School's Out—Forever (Maximum Ride #2),3,5.0
3,A Fistful of Charms (The Hollows #4),2,5.0
4,A Woman of Substance (Emma Harte Saga #1),2,5.0
5,Act of Treason (Mitch Rapp #9),2,5.0
6,Alas Babylon,2,5.0
7,Angels Fall,2,5.0
8,Arrows of the Queen (Heralds of Valdemar #1),2,5.0
9,Captivating: Unveiling the Mystery of a Woman'...,2,5.0


<a id="task_three"></a> 
#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [12]:
query_one = '''SELECT    p.publisher as publisher,
                         count(b.book_id) as book_count
               FROM      publishers as p
               LEFT JOIN books as b on b.publisher_id = p.publisher_id
               WHERE     b.num_pages > 50
               GROUP BY  publisher
               ORDER BY  book_count desc
               LIMIT     1
             ''' 

con=engine.connect() 
pd.io.sql.read_sql(sql=text(query_one), con = con)

,publisher,book_count
0,Penguin Books,42


Издательство 'Penguin Books' выпустило наибольшее число книг с количеством страниц больше 50  

<a id="task_four"></a> 
#### Определим  автора с самой высокой средней оценкой книг , будем  учитывать только книги с 50 и более оценками

In [13]:
query_one = '''with 
               one as (SELECT    book_id,
                                 round(avg(rating),2) as rating_avg
                       FROM      ratings
                       GROUP BY  book_id
                       HAVING    COUNT(rating_id) >= 50
                       ORDER BY  rating_avg desc),
             
               two as (SELECT      b.author_id,
                                   avg(one.rating_avg) as rating_author_avg
                       FROM        books as b
                       INNER JOIN  one on one.book_id = b.book_id
                       GROUP BY    author_id
                       ORDER BY    rating_author_avg  desc
                       LIMIT       1)
               SELECT     a.author,
                          round(two.rating_author_avg,2)
               FROM       authors as a
               INNER JOIN two on two.author_id = a.author_id           
  
            ''' 
con=engine.connect() 
pd.io.sql.read_sql(sql=text(query_one), con = con)

,author,round
0,J.K. Rowling/Mary GrandPré,4.29


J.K. Rowling/Mary GrandPre - дуэт авторов с самой высокой средней оценкой книг 4.29

<a id="task_five"></a> 
#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок

In [14]:
query_one = '''SELECT AVG(rev_count) 
               FROM   (SELECT   username,
                                COUNT(review_id) as rev_count
                       FROM     reviews
                       WHERE    username in (SELECT    username
                                             FROM      ratings
                                             GROUP BY  username
                                             HAVING    COUNT(rating_id) > 48)
                       GROUP BY username) as five
                            
             ''' 
con=engine.connect() 
pd.io.sql.read_sql(sql=text(query_one), con = con)

,avg
0,24.0


Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно 24

<a id="conclusion"></a>
### Вывод

__Цель__ нашего проекта заключалась в том, чтобы провести анализ базы данных сервиса для чтения книг по подписке.

__Задачи__ проекта:
1. Посчитать, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитать количество обзоров и среднюю оценку;
3. Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры;
4. Определить автора с самой высокой средней оценкой книг, учитывая только книги с 50 и более оценками;
5. Посчитать среднее количество обзоров от пользователей, которые поставили больше 48 оценок.


В нашем распоряжении пять таблиц:
* таблица books содержит данные о книгах;
* таблица authors содержит данные об авторах;
* таблица publishers содержит данные об издательствах;
* таблица ratings содержит данные о пользовательских оценках книг;
* таблица reviews содержит данные о пользовательских обзорах

Работу выполнили в два этапа.

На __первом этапе__ загрузили и познакомились с данными

На __втором этапе__  мы провели анализ данных и выяснили следующее:

1. После 1 января 2000 года вышло __819__ книг
2. Для каждой книги посчитали количество обзоров и среднюю оценку
3. Издательство __'Penguin Books'__ выпустило наибольшее число книг с количеством страниц 50 и больше 
4. __J.K. Rowling/Mary GrandPre__ - дуэт авторов с самой высокой средней оценкой книг
5. Среднее количество обзоров от пользователей, которые поставили больше 48 оценок, равно __24__ .

